# core 

> modifying streamlit methods to play nice with jupyter

Currently streamlit expects to find itself in a standalone python script, and will not work in a jupyter notebook. This is because streamlit is a web app, and jupyter notebooks are not.

To get around this, we can use the `get_ipython` method to check if we are in a jupyter notebook, and if so, we can use the `display` method to display the streamlit app in the notebook.

This allows us to interactively code in our jupyter notebook and see how our streamlit app will look in real time.

In [1]:
# | default_exp core 

In [1]:
# | export

import logging 
import time
import typing as tp

import streamlit
from fastcore.basics import in_ipython, listify, noop
from fastcore.test import test_fail
from IPython.utils.capture import capture_output


In [2]:
# | export

# module obljects that we will be importing
IN_IPYTHON = in_ipython()

In [3]:
assert IN_IPYTHON, "This module is intended to be used in a Jupyter notebook"

In [29]:
# | export
import inspect

class StreamlitPatcher:
    """class to patch streamlit functions for displaying content in jupyter notebooks"""
    def __init__(self):
      pass

    def patch(self):
        """
        Registers the current `tqdm` class with
            streamlit.
            ( write 
              markdown

            )
            """

        pass

        # patch streamlit

        # patch stqdm

    def echo(self):
      print("hello")

    @property
    def methods(self):
      for attr in dir(self):
          if attr == "methods":
            continue

          m = getattr(self, attr)
          if inspect.ismethod(m) and not attr.startswith("_"):
              print(attr)

    @staticmethod
    def _get_streamlit_methods():
      """get all streamlit methods"""
      return [attr for attr in dir(streamlit) if not attr.startswith("_")]

In [30]:
sp = StreamlitPatcher()

In [31]:
getattr(streamlit, "write")

<function streamlit.delta_generator.WriteMixin.write(self, *args: Any, unsafe_allow_html: bool = False, **kwargs) -> None>

In [12]:
import functools
import IPython.display


def _display(arg: tp.Any) -> None:
    if isinstance(arg, str):
        IPython.display.display(IPython.display.Markdown(arg))
    else:
        IPython.display.display(arg)


def _st_jupyter(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        # do something before invocation
        for arg in args:
            _display(arg)

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


# st.write = _st_jupyter(st.write)

In [23]:
method_name = "write"
trg = getattr(streamlit, method_name)
wrapper = _st_jupyter

setattr(streamlit, method_name, wrapper(trg))

hello

In [28]:
with capture_output() as cap:
    streamlit.write("hello")
    got = cap._outputs[0]["data"]

expected = {'text/plain': '<IPython.core.display.Markdown object>',
 'text/markdown': 'hello'} 
assert got == expected, "check that the output is correct"

In [27]:
got

{'text/plain': '<IPython.core.display.Markdown object>',
 'text/markdown': 'hello'}

In [11]:
StreamlitPatcher().methods

echo
patch


echo
patch


## Modifying streamlit

The way we will modify streamlit methods is by putting them through a decorator. This decorator will check if we are in a jupyter notebook, and if so, it will take the input and display it in the notebook. 

Else it will use the original streamlit method.

In [ ]:
if IN_IPYTHON:
    from tqdm.notebook import tqdm
else:
    from stqdm import stqdm as tqdm

tqdm = tqdm  # make this available in the module namespace

In [ ]:
for i in tqdm(range(10)):
    pass

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import functools

import IPython.display


def _display(arg: tp.Any) -> None:
    if isinstance(arg, str):
        IPython.display.display(IPython.display.Markdown(arg))
    else:
        IPython.display.display(arg)


def _st_jupyter(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        # do something before invocation
        for arg in args:
            _display(arg)

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


st.write = _st_jupyter(st.write)

In [ ]:
st.write("This is **bold** text in markdown")

This is **bold** text in markdown

In [ ]:
try:
    import pandas as pd
    df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
    st.write(df)
except ImportError:
    logger.warning("Pandas not installed, skipping test")

,a,b
0,1,4
1,2,5
2,3,6


In [ ]:
def _patch_st_heading(func_to_decorate, tag):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) == 2:
            body, anchor = args
        elif len(args) > 2:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if "body" not in kwargs:
                raise ValueError(
                    f"Missing required argument: body, {func_to_decorate.__name__} requires a body"
                )
            body = kwargs["body"]

        if isinstance(body, str):
            _display(f"{tag} {body}")
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts strings"
            )

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


st.title = _patch_st_heading(st.title, "#")
st.header = _patch_st_heading(st.header, "##")
st.subheader = _patch_st_heading(st.subheader, "###")

In [ ]:
st.title("This is a title")

# This is a title

In [ ]:
st.header("This is a header")

## This is a header

In [ ]:
st.subheader("This is a subheader")

### This is a subheader

In [ ]:
# these should fail

test_fail(lambda: st.title(df), contains="Unsupported type")
test_fail(lambda: st.header(df), contains="Unsupported type")
test_fail(lambda: st.subheader(df), contains="Unsupported type")
test_fail(lambda: st.subheader(1), contains="Unsupported type")

In [ ]:
def _wrap_st_type_check(
    func_to_decorate: tp.Callable, allowed_types: tp.Type | tp.Collection[tp.Type]
) -> tp.Callable:
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""
    allowed_types = listify(allowed_types)  # make sure it's a list

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 1:
            body = args[0]
        elif len(args) > 1:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )
        elif len(args) == 0:
            if kwargs:
                raise NotImplementedError(
                    f"kwargs not supported yet, 'streamlit_data_science.utils._wrap_st_type_check' only accepts positional arguments"
                )
            else:
                raise ValueError(f"at least one positional argument is required")

        if type(body) in allowed_types:
            _display(body)
        else:
            raise TypeError(
                f"Unsupported type: {type(body)}, {func_to_decorate.__name__} only accepts {allowed_types}"
            )

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


st.markdown = _wrap_st_type_check(st.markdown, str)

In [ ]:
test_fail(lambda: st.markdown(df), contains="Unsupported type")

st.markdown("This is **bold** text in markdown")

This is **bold** text in markdown

## st.cache

The `st.cache` method is used to cache the output of a function. This is useful for functions that take a long time to run, and we want to avoid running them every time we run the app.

If we are in a jupyter notebook, we can't use the `st.cache` method, so we will replace the `st.cache` method with a dummy method that does nothing.

In [ ]:


def _dummy_wrapper_noop(func_to_decorate):
    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        return noop  # castrate the function to do nothing

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


st.cache = _dummy_wrapper_noop(st.cache)

In [ ]:
# verify that during patching we didn't change the name or docstring
assert st.cache.__name__ == "cache"
assert "@st.cache" in tp.cast(
    str, st.cache.__doc__
), "check that the docstring is correct"

In [ ]:
# test caching


@st.cache(suppress_st_warning=True)
def get_data():
    st.write("Getting data...")
    for i in tqdm(range(5)):
        time.sleep(0.1)
    return pd.DataFrame({"c": [7, 8, 9], "d": [10, 11, 12]})


df = get_data()
st.write(df)

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

,c,d
0,7,10
1,8,11
2,9,12


In [ ]:
# test that the cache in jupyter does not affect get_data

df = get_data()
with capture_output() as cap:
    st.write(df)
    got = cap._outputs[0]["data"]

expected = {
    "text/plain": "   c   d\n0  7  10\n1  8  11\n2  9  12",
    "text/html": '<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-align: middle;\n    }\n\n    .dataframe tbody tr th {\n        vertical-align: top;\n    }\n\n    .dataframe thead th {\n        text-align: right;\n    }\n</style>\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>c</th>\n      <th>d</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>7</td>\n      <td>10</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>8</td>\n      <td>11</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>9</td>\n      <td>12</td>\n    </tr>\n  </tbody>\n</table>\n</div>',
}

assert got == expected, "check that the output is correct"

Getting data...

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:

# patch st.expander to display in jupyter


class _DummyExpander:
    def __init__(self, label: str, expanded: bool = False):
        self.label = label
        self.expanded = expanded

    def __enter__(self):
        _display(f">**expander starts**: {self.label}")

    def __exit__(self, *args):
        _display(f">**expander ends**")
        pass


if IN_IPYTHON:
    st.expander = _DummyExpander  # type: ignore

In [ ]:
with st.expander("Expand me!"):
    st.markdown(
        """
    The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.
    
    Particularly, you'll be able to:
    - Set up a coding environment for building Streamlit apps
    - Build your first Streamlit app
    - Learn about all the awesome input/output widgets to use for your Streamlit app
    """
    )

>**expander starts**: Expand me!


    The **#30DaysOfStreamlit** is a coding challenge designed to help you get started in building Streamlit apps.
    
    Particularly, you'll be able to:
    - Set up a coding environment for building Streamlit apps
    - Build your first Streamlit app
    - Learn about all the awesome input/output widgets to use for your Streamlit app
    

>**expander ends**

## Interactive Methods


In [ ]:
def _st_text_input(func_to_decorate):
    """Decorator to display objects passed to Streamlit in Jupyter notebooks."""

    @functools.wraps(func_to_decorate)
    def wrapper(*args, **kwargs):
        if len(args) == 2:
            label, default = args
        elif len(args) == 1:
            label = args[0]
            default = kwargs["value"] if kwargs else input(f"{label}: ")
        else:
            raise ValueError(
                f"Too many positional arguments: {len(args)}, {func_to_decorate.__name__} only accepts 2"
            )

        st.write(
            f"""```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   {label}: {default:<21} #
############################################        
"""
        )

    if IN_IPYTHON:
        return wrapper
    else:
        return func_to_decorate


st.text_input = _st_text_input(st.text_input)

In [ ]:
st.text_input("Enter some text", "default text")

```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   Enter some text: default text          #
############################################        


In [ ]:
st.text_input("Enter some text", value="foobar")

```
############################################        
#       TEXT INPUT FIELD                   #
#                                          #
#   Enter some text: foobar                #
############################################        


In [ ]:
# | notest
# st.text_input("Enter some text")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()